In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 4.4.10
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base conda




Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 4.4.10
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base conda




Libraries imported.


In [2]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [4]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 0 boroughs and 0 neighborhoods.


In [5]:
address = 'Milan, IT'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Milan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Milan are 45.4668, 9.1905.


In [6]:
# create map of Milan using latitude and longitude values
map_milan = folium.Map(location=[latitude, longitude], zoom_start=10)    
map_milan

In [16]:
CLIENT_ID = 'NEG0T1NXGRAAAH54DI4RC4DW0FT2W0B4BMLI5VD4VLJFINPS' # your Foursquare ID
CLIENT_SECRET = '2TGRSZJZ22PAQPSGHM3IERE5JVN0YGY0ZSZV5FFZ0PQ5405A' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NEG0T1NXGRAAAH54DI4RC4DW0FT2W0B4BMLI5VD4VLJFINPS
CLIENT_SECRET:2TGRSZJZ22PAQPSGHM3IERE5JVN0YGY0ZSZV5FFZ0PQ5405A


In [17]:
search_query = 'Sushi'
radius = 2000
print(search_query + ' .... OK!')

Sushi .... OK!


In [18]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=NEG0T1NXGRAAAH54DI4RC4DW0FT2W0B4BMLI5VD4VLJFINPS&client_secret=2TGRSZJZ22PAQPSGHM3IERE5JVN0YGY0ZSZV5FFZ0PQ5405A&ll=45.4668,9.1905&v=20180605&query=Sushi&radius=2000&limit=100'

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ee4a48c226d4f12e8ad43de'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/sushi_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d1d2941735',
      'name': 'Sushi Restaurant',
      'pluralName': 'Sushi Restaurants',
      'primary': True,
      'shortName': 'Sushi'}],
    'hasPerk': False,
    'id': '4b9b9343f964a520a81036e3',
    'location': {'address': 'Piazza Duomo',
     'cc': 'IT',
     'city': 'Milano',
     'country': 'Italia',
     'crossStreet': 'Piazza Duomo',
     'distance': 238,
     'formattedAddress': ['Piazza Duomo (Piazza Duomo)',
      '20100 Milano Lombardia',
      'Italia'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 45.46485720004078,
       'lng': 9.191785645372121}],
     'lat': 45.46485720004078,
     'lng': 9.191785645372121,
     'neighborhood': 'Duomo',
     'postalCode': '20100',
     'state': 'Lombardia'},
    'name': 'MySushi Conveyor',

In [19]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,4b9b9343f964a520a81036e3,Piazza Duomo,IT,Milano,Italia,Piazza Duomo,238,"[Piazza Duomo (Piazza Duomo), 20100 Milano Lom...","[{'label': 'display', 'lat': 45.46485720004078...",45.464857,9.191786,Duomo,20100,Lombardia,MySushi Conveyor,v-1592042786
1,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",False,4c80e268d8086dcb35456752,NaN,IT,NaN,Italia,NaN,453,[Italia],"[{'label': 'display', 'lat': 45.46601405731972...",45.466014,9.184796,NaN,NaN,NaN,Sushi Drink,v-1592042786
2,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",False,510fb689e4b04d3fa27b7b84,NaN,IT,NaN,Italia,NaN,627,[Italia],"[{'label': 'display', 'lat': 45.46168068619187...",45.461681,9.187128,NaN,NaN,NaN,Sushi My Food Store,v-1592042786
3,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,5c705dfffc9e94002c73e22c,8 Via Spadari,IT,Milano,Italia,NaN,437,"[8 Via Spadari, 20123 Milano Lombardia, Italia]","[{'label': 'display', 'lat': 45.463772, 'lng':...",45.463772,9.186927,NaN,20123,Lombardia,Balada Sushi,v-1592042786
4,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",False,53b2e907498ec970f3e48dbd,"Via Fiori Chiari, 1/A",IT,Milano,Italia,Via Brera,659,"[Via Fiori Chiari, 1/A (Via Brera), 20121 Mila...","[{'label': 'display', 'lat': 45.47215281771100...",45.472153,9.186883,NaN,20121,Lombardia,SUSHI B,v-1592042786
5,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",False,4e71e0f81f6ecfe82bfeee30,NaN,IT,Milano,Italia,NaN,506,"[Milano Lombardia, Italia]","[{'label': 'display', 'lat': 45.462258, 'lng':...",45.462258,9.190219,NaN,NaN,Lombardia,Degustazione Bento Sushi - Downtown Diaz,v-1592042786
6,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,5676e3a4498e64d8f6a66edf,via degli arcimboldi,IT,Milano,Italia,via dell'unione,595,"[via degli arcimboldi (via dell'unione), 20122...","[{'label': 'display', 'lat': 45.46197188124476...",45.461972,9.187222,NaN,20122,Lombardia,sushili,v-1592042786


In [20]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,MySushi Conveyor,Sushi Restaurant,Piazza Duomo,IT,Milano,Italia,Piazza Duomo,238,"[Piazza Duomo (Piazza Duomo), 20100 Milano Lom...","[{'label': 'display', 'lat': 45.46485720004078...",45.464857,9.191786,Duomo,20100,Lombardia,4b9b9343f964a520a81036e3
1,Sushi Drink,Japanese Restaurant,NaN,IT,NaN,Italia,NaN,453,[Italia],"[{'label': 'display', 'lat': 45.46601405731972...",45.466014,9.184796,NaN,NaN,NaN,4c80e268d8086dcb35456752
2,Sushi My Food Store,Japanese Restaurant,NaN,IT,NaN,Italia,NaN,627,[Italia],"[{'label': 'display', 'lat': 45.46168068619187...",45.461681,9.187128,NaN,NaN,NaN,510fb689e4b04d3fa27b7b84
3,Balada Sushi,Sushi Restaurant,8 Via Spadari,IT,Milano,Italia,NaN,437,"[8 Via Spadari, 20123 Milano Lombardia, Italia]","[{'label': 'display', 'lat': 45.463772, 'lng':...",45.463772,9.186927,NaN,20123,Lombardia,5c705dfffc9e94002c73e22c
4,SUSHI B,Japanese Restaurant,"Via Fiori Chiari, 1/A",IT,Milano,Italia,Via Brera,659,"[Via Fiori Chiari, 1/A (Via Brera), 20121 Mila...","[{'label': 'display', 'lat': 45.47215281771100...",45.472153,9.186883,NaN,20121,Lombardia,53b2e907498ec970f3e48dbd
5,Degustazione Bento Sushi - Downtown Diaz,Food,NaN,IT,Milano,Italia,NaN,506,"[Milano Lombardia, Italia]","[{'label': 'display', 'lat': 45.462258, 'lng':...",45.462258,9.190219,NaN,NaN,Lombardia,4e71e0f81f6ecfe82bfeee30
6,sushili,Restaurant,via degli arcimboldi,IT,Milano,Italia,via dell'unione,595,"[via degli arcimboldi (via dell'unione), 20122...","[{'label': 'display', 'lat': 45.46197188124476...",45.461972,9.187222,NaN,20122,Lombardia,5676e3a4498e64d8f6a66edf


In [46]:
# create map of Milan using latitude and longitude values
map_milan = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng,in zip(dataframe_filtered['lat'], dataframe_filtered['lng']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_milan)  
    
map_milan

In [47]:
dataframe_filtered.name

0    MySushi Conveyor                        
1    Sushi Drink                             
2    Sushi My Food Store                     
3    Balada Sushi                            
4    SUSHI B                                 
5    Degustazione Bento Sushi - Downtown Diaz
6    sushili                                 
Name: name, dtype: object

In [50]:
venue_id = '4b9b9343f964a520a81036e3' # ID of My Sushi Conveyor
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4b9b9343f964a520a81036e3?client_id=NEG0T1NXGRAAAH54DI4RC4DW0FT2W0B4BMLI5VD4VLJFINPS&client_secret=2TGRSZJZ22PAQPSGHM3IERE5JVN0YGY0ZSZV5FFZ0PQ5405A&v=20180605'

In [51]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'description', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'parent', 'hierarchy', 'attributes', 'bestPhoto', 'colors'])


{'allowMenuUrlEdit': True,
 'attributes': {'groups': [{'count': 1,
    'items': [{'displayName': 'Price', 'displayValue': '€€€', 'priceTier': 3}],
    'name': 'Price',
    'summary': '€€€',
    'type': 'price'},
   {'count': 3,
    'items': [{'displayName': 'Reservations', 'displayValue': 'Yes'}],
    'name': 'Reservations',
    'summary': 'Reservations',
    'type': 'reservations'},
   {'count': 5,
    'items': [{'displayName': 'Credit Cards', 'displayValue': 'Yes'}],
    'name': 'Credit Cards',
    'summary': 'Credit Cards',
    'type': 'payments'},
   {'count': 1,
    'items': [{'displayName': 'Outdoor Seating', 'displayValue': 'No'}],
    'name': 'Outdoor Seating',
    'type': 'outdoorSeating'},
   {'count': 8,
    'items': [{'displayName': 'Brunch', 'displayValue': 'Brunch'},
     {'displayName': 'Lunch', 'displayValue': 'Lunch'},
     {'displayName': 'Dinner', 'displayValue': 'Dinner'}],
    'name': 'Menus',
    'summary': 'Dinner, Lunch & more',
    'type': 'serves'},
   {'count

In [52]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.4


In [53]:
result['response']['venue']['tips']['count']

15

In [54]:
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ee4aab9867c843e1a78af45'},
 'response': {'tips': {'count': 15,
   'items': [{'agreeCount': 0,
     'authorInteractionType': 'liked',
     'canonicalUrl': 'https://foursquare.com/item/54a99b0d498e56390e5ad5cf',
     'createdAt': 1420401421,
     'disagreeCount': 0,
     'id': '54a99b0d498e56390e5ad5cf',
     'lang': 'it',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'photo': {'createdAt': 1420401421,
      'height': 540,
      'id': '54a99b0d498e56390e5ad5ce',
      'prefix': 'https://fastly.4sqi.net/img/general/',
      'source': {'name': 'Swarm for Android',
       'url': 'https://www.swarmapp.com'},
      'suffix': '/5915976_v_dv98k1Z4DsOUsJu5Aer_YSipyG5uZtllbbhLJQT0E.jpg',
      'visibility': 'public',
      'width': 960},
     'photourl': 'https://fastly.4sqi.net/img/general/original/5915976_v_dv98k1Z4DsOUsJu5Aer_YSipyG5uZtllbbhLJQT0E.jpg',
     'text': 'Sushi di alto livello. Molto cordiali. Degustate i piattini con il 

In [55]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'photo', 'photourl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

In [31]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

C:\Users\francesco traversa\Anaconda3\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,Sushi di alto livello. Molto cordiali. Degustate i piattini con il sake caldo. Però occhio: costa un botto.,0,0,54a99b0d498e56390e5ad5cf,Silvio,D,NaN,5915976
